## Generating GCG Suffixes Using Azure Machine Learning 

This notebook shows how to generate GCG suffixes using Azure Machine Learning (AML), which consists of three main steps:
1. Connect to an Azure Machine Learning (AML) workspace.
2. Create AML Environment with the Python dependencies.
3. Submit a training job to AML.

### Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning (AML), providing a centralized place to work with all the artifacts you create when using AML. In this section, we will connect to the workspace in which the job will be run.

To connect to a workspace, we need identifier parameters - a subscription, resource group and workspace name. We will use these details in the `MLClient` from `azure.ai.ml` to get a handle to the required AML workspace. We use the [default Azure authentication](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity.defaultazurecredential?view=azure-python) for this tutorial. Check the [configuration notebook](../../../configuration.ipynb) for more details on how to configure credentials and connect to a workspace.

In [1]:
import os
from pyrit.common import default_values

default_values.load_default_env()

# Enter details of your AML workspace
subscription_id = os.environ.get('AZURE_SUBSCRIPTION_ID')
resource_group = os.environ.get('AZURE_RESOURCE_GROUP')
workspace = os.environ.get('AZURE_ML_WORKSPACE_NAME')

In [2]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

# Get a handle to the workspace
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

### Create AML Environment

To install the dependencies needed to run GCG, we create an AML environment from a [Dockerfile](../../../pyrit/auxiliary_attacks/gcg/src/Dockerfile).

In [3]:
from azure.ai.ml.entities import Environment, BuildContext

# Configure the AML environment with path to Dockerfile and dependencies
env_docker_context = Environment(
    build=BuildContext(path="../../../pyrit/auxiliary_attacks/gcg/src"),
    name="pyrit",
    description="PyRIT environment created from a Docker context.",
)

# Create or update the AML environment
ml_client.environments.create_or_update(env_docker_context)

Environment({'intellectual_property': None, 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'pyrit', 'description': 'PyRIT environment created from a Docker context.', 'tags': {}, 'properties': {'azureml.labels': 'latest'}, 'print_as_yaml': True, 'id': '/subscriptions/6d13156d-cc60-485a-9e9e-54857689f99b/resourceGroups/romanlutz-southcentralus/providers/Microsoft.MachineLearningServices/workspaces/romanlutz-southcentralus/environments/pyrit/versions/31', 'Resource__source_path': None, 'base_path': 'C:\\Users\\bbullwinkel\\OneDrive - Microsoft\\Documents\\AIRT\\PyRIT\\doc\\code\\auxiliary_attacks', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x0000018D06B31FD0>, 'serialize': <msrest.serialization.Serializer object at 0x0000018D07576E10>, 'version': '31', 'latest_version': None, 'conda_file': None, 'image': None, 'build': <azure.ai.ml.entities._assets.environment.BuildContext object at 0x0000018D06F56990>, 'infe

### Submit Training Job to AML

Finally, we configure the command to run the GCG algorithm. The entry file for the algorithm is []`run.py`](../../../pyrit/auxiliary_attacks/gcg/experiments/run.py), which takes several command line arguments, as shown below. We also have to specify the compute `instance_type` to run the algorithm on. In our experience, a GPU instance with at least 32GB of vRAM is required. As shown below, we use Standard_ND40rs_v2.

In [4]:
from azure.ai.ml import command
from azure.ai.ml.entities import JobResourceConfiguration

# Configure the command
job = command(
    code="./../../..",  # local path where the code is stored
    command="cd pyrit/auxiliary_attacks/gcg/experiments && python run.py --model_name ${{inputs.model_name}} --setup ${{inputs.setup}} --n_train_data ${{inputs.n_train_data}} --n_test_data ${{inputs.n_test_data}} --n_steps ${{inputs.n_steps}} --batch_size ${{inputs.batch_size}}",
    inputs={
        "model_name": "phi_3_mini",
        "setup": "multiple",
        "n_train_data": 25,
        "n_test_data": 0,
        "n_steps": 500,
        "batch_size": 512,
    },
    environment=f"{env_docker_context.name}:{env_docker_context.version}",
    environment_variables={"HF_TOKEN": os.environ["HF_TOKEN"]},
    display_name="suffix_generation",
    description="Generate a suffix for attacking LLMs.",
    resources = JobResourceConfiguration(
        instance_type="Standard_ND40rs_v2",
        instance_count=1,
    )
)

In [5]:
# Submit the command
returned_job = ml_client.create_or_update(job)

Class AutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


Class AutoDeleteConditionSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


Class BaseAutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


Class IntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


Class ProtectionLevelSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


...

Uploading PyRIT (55.49 MBs):  28%|######################################8                                                                                                     | 15401756/55493716 [00:13<00:16, 2427732.14it/s]

Uploading PyRIT (55.49 MBs):  35%|################################################7                                                                                           | 19322980/55493716 [00:15<00:15, 2260682.00it/s]

Uploading PyRIT (55.49 MBs):  44%|############################################################9                                                                               | 24169614/55493716 [00:15<00:08, 3878205.46it/s]

Uploading PyRIT (55.49 MBs): 100%|############################################################################################################################################| 55493716/55493716 [00:29<00:00, 2432861.91it/s]

Uploading PyRIT (55.49 MBs): 100%|############################################################################################################################################| 55493716/55493716 [00:29<00:00, 1894859.90it/s]